<a href="https://colab.research.google.com/github/KeyboarderSon/TIL/blob/main/RNN/RNN_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index={}
    self.word2count={}
    self.index2word={0:"SOS", 1:"EOS"}
    self.n_words=2 # SOS , EOS

In [ ]:
# main reference
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# https://github.com/deeplearningzerotoall/PyTorch/blob/master/RNN/6-seq2seq.py


import random
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#sequence : 문장
#token : 단어

raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

# Start Of Sentence, End Of Sentence
SOS_token = 0
EOS_token = 1


# class for vocabulary related information of data
class Vocab:
  def __init__(self):
    self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}
    self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
    self.vocab_count = {}
    self.n_vocab = len(self.vocab2index)

  def add_vocab(self, sentence):
    for word in sentence.split(" "):
      if word not in self.vocab2index:
        self.vocab2index[word] = self.n_vocab
        self.vocab_count[word] = 1
        self.index2vocab[self.n_vocab] = word
        self.n_vocab += 1
      else:#if word in self.vocab2index
        self.vocab_count[word] += 1


# source text, target text 분리
# 데이터가 잘 학습되도록 준비하는 전처리과정
def preprocess(corpus, source_max_length, target_max_length):
  print("reading corpus...")
  pairs = []
  for line in corpus:
    pairs.append([s for s in line.strip().lower().split("\t")])
  print("Read {} sentence pairs".format(len(pairs)))
  

## 나동빈 seq2seq

spaCy 라이브러리 : 문장의 토큰화, 태깅 등의 전처리 기능을 위한 라이브러리

In [ ]:
%%capture

!python -m spacy download en
!python -m spacy download de

In [ ]:
import  spacy
#영어 및 독일어 토큰화
spacy_en= spacy.load('en')
spacy_de= spacy.load('de')

In [ ]:
#간단하게 토큰화 기능 사용해보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


영어 및 독일어 토큰화 함수 정의

In [ ]:
def tokenize_de(text):
  #토큰화한 뒤 reverse it
  return [token.text for token in spacy_de.tokenizer(text)][::-1]


def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

field 라이브러리를 이용해 각 문장에 대한 구체적인 전처리 내용을 명시<br>
Source 독일어 -> Target 영어

In [ ]:
from torchtext.data import Field, BucketIterator
#lower : 소문자로 처리하도록
SRC=Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)
TRG=Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)

In [ ]:
from torchtext.datasets import Multi30k

#각 문장을 Field에 정의된 내용을 기반으로 토큰화
train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 803kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 274kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 265kB/s]


In [ ]:
print(f"training dataset 크기 : {len(train_dataset.examples)}")
print(f"validation dataset 크기 : {len(valid_dataset.examples)}")
print(f"testing dataset 크기 : {len(test_dataset.examples)}")

training dataset 크기 : 29000
validation dataset 크기 : 1014
testing dataset 크기 : 1000


In [ ]:
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


필드(field) 객체의 build_vocab 메서드를 이용해 영독 단어 사전을 생성합니다.<br>
최소 2번 이상 등장한 단어만을 선택합니다.

```SRC.vocab.stoi``` 는 어휘에 해당하는 토큰을 키로, 관련된 색인을 값으로 가지는 사전(dict)이 된다. 토큰 : 색인<br>


In [ ]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7855
len(TRG): 5893


In [ ]:
print(TRG.vocab.stoi["abcabc"])#없는 단어 : 0
print(TRG.vocab.stoi[TRG.pad_token])
print(TRG.vocab.stoi["<sos>"])
print(TRG.vocab.stoi["<eos>"])
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


한 문장에 포함된 단어가 연속적으로 LSTM에 입력되어야 합니다. 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋습니다.<br> 이를 위해 BucketIterator를 사용합니다.
<br>batch size = 128

모든 텍스트 작업을 일괄 처리하고 단어를 인덱스 숫자로 변환하는 것을 돕습니다. 이때 

In [ ]:
import torch

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE=128

train_iterator, valid_iterator, test_iterator=BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device
)

In [ ]:
for i, batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  print(f"첫 번째 배치 크기 : {src.shape}")

  for i in range(src.shape[0]):
    print(f"idx {i}: {src[i][0].item()}")
  
  break

첫 번째 배치 크기 : torch.Size([28, 128])
idx 0: 2
idx 1: 4
idx 2: 6200
idx 3: 1439
idx 4: 8
idx 5: 417
idx 6: 6
idx 7: 11
idx 8: 89
idx 9: 2701
idx 10: 14
idx 11: 7
idx 12: 16
idx 13: 14
idx 14: 22
idx 15: 886
idx 16: 0
idx 17: 5
idx 18: 3
idx 19: 1
idx 20: 1
idx 21: 1
idx 22: 1
idx 23: 1
idx 24: 1
idx 25: 1
idx 26: 1
idx 27: 1


Encoder architecture
주어진 소스 문자을 context vector로 인코딩합니다.<br>
LSTM은 hidden state과 cell state을 반환합니다.<br>
**Hyperparameter**<br>
input_dim : 하나의 단어에 대한 one hot encoding 차원<br>
embed_dim : 임베딩(embeding} 차원<br>
hidden_state<br>
n_layers : RNN 레이어의 갯수<br>
dropout_ratio

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
    super().__init__()

    #embedding? one-hot encoding을 특정 차원의 임베딩으로 매핑하는 레이어
    self.embedding = nn.Embedding(input_dim, embed_dim)

    #LSTM 레이어
    self.hidden_dim=hidden_dim
    self.n_layers=n_layers
    self.rnn=nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
    #input이 embeding을 거쳐 들어가기에 embed_dim만 존재,
    #return은 cell state, hidden state

    #dropout
    self.dropout=nn.Dropout(dropout_ratio)

  #인코더는 소스 문장을 입력으로 받아 context vector을 반환
  def forward(self, src):
    
    embedded = self.dropout(self.embedding(src))
    # src[단어개수, batch size] -> embedded[단어개수, batch size, embedding dim]

    output, (hidden, cell) = self.rnn(embedded)
    #output[단어개수, batch size, hidden dim] : 현재 단어의 출력 정보
    #hidden[n_layer, batch size, hidden dim] : 현재까지의 모든 단어의 정보
    #cell[n_layer, batch size, hidden dim] : 현재 까지의 모든 단어의 정보

    #  *** hidden과 cell의 차이는 그럼 뭐지? ***

    # context vector 반환
    return hidden, cell


### Decoder

context vector을 타겟 문장으로 디코딩한다.
LSTM은 hidden state와 cell state를 반환한다.

Hyperparameter<br>
input dim<br>
embed dim<br>
hidden dim<br>
n layers<br>
dropout ratio

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
    super().__init__()

    self.embedding=nn.embedding(output_dim, embed_dim)
    